# Implementing a DCGAN: Image Generation with Deep Convolutional GANs

In this lab, you will construct a Deep Convolutional Generative Adversarial Network (DCGAN). Key concepts covered include:

* **Convolutional Architectures for Image Generation:** Discover why convolutional layers excel at image-related tasks.
* **Generative Adversarial Networks (GANs):**  Understand the adversarial training process between the generator and discriminator networks.

Let's proceed with implementing your DCGAN.

## Data Preparation

**Dataset:**   We'll use a grayscale dataset (Fashion-MNIST) for this exercise. To speed up computation (these networks can be quite computationally intensive), we'll resize the images to 16x16 pixels.

In [ ]:
import tensorflow as tf
from skimage.transform import resize

# Load and Prepare Fashion-MNIST Dataset
(train_images, train_labels), (_, _) = tf.keras.datasets.fashion_mnist.load_data()

# Resize Images
resized_images = resize(train_images, (train_images.shape[0], 16, 16, 1),
                        preserve_range=True).astype("float32")

# Normalize Pixel Values
normalized_images = (resized_images - 127.5) / 127.5

# Verify Shape  
print(normalized_images.shape)

In [ ]:
data_generator = tf.data.Dataset.from_tensor_slices(
    normalized_images).shuffle(60000).batch(100, drop_remainder=True)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def display_images(images, rows=2, cols=10, figsize=(20, 3)):
    fig = plt.figure(figsize=figsize)
    for i in range(rows * cols):
        ax = plt.subplot(rows, cols, i + 1)
        ax.set_axis_off()
        ax.imshow(images[i, :, :, 0], cmap='gray')

    return fig

In [ ]:
display_images(normalized_images)

## Architecting the Generator and Discriminator

Let's design the core components of the DCGAN.

### Generator: Crafting Images from Noise

The generator transforms random noise into realistic images. This network typically consists of:

* **Input:**  Random noise (e.g., 100 dimensions).
* **Series of Transposed Convolutional Layers:** Upsample the noise into a tensor that resembles an image.  A typical structure might be:
    1. 512 channels, kernel size 4, stride 1, 'valid' padding, BatchNorm, LeakyReLU
    2. 256 channels, kernel size 4, stride 2, 'same' padding, BatchNorm, LeakyReLU
    3. 128 channels, kernel size 4, stride 1, 'same' padding, BatchNorm, LeakyReLU
* **Output:**  A 16x16x1 image.

### Understanding Transposed Convolutions

The transposed convolution (also known as a deconvolution) is the key to the generator's ability to upsample the noise into an image.  It is the opposite of a standard convolution.  Given an input, it will increase the dimensions by padding the input with zeros and then applying a convolution.  This is why it is sometimes called an "upsampling" layer.

### Batch Normalization

Batch normalization is a technique to improve the training of deep neural networks.  It works by normalizing the input of each layer, meaning that the input to each layer has a mean of zero and a standard deviation of one.  This helps to stabilize and speed up training.

### LeakyReLU

LeakyReLU is a variant of the ReLU activation function.  It has a small slope for negative values, which can help to prevent the "dying ReLU" problem. This problem occurs when a ReLU neuron always outputs the same value (e.g., 0), effectively "dying" and ceasing to learn.

### Discriminator: Discerning Real from Fake

The discriminator learns to distinguish between real images and the generator's creations.   A typical structure might be:

* **Input:**  Image data.
* **Series of Convolutions:** Extract features, as we are familiar with from image classification tasks.  A typical structure might be:
    1. 128 channels, kernel size 4, stride 2, 'same' padding, BatchNorm, LeakyReLU
    1. 256 channels, kernel size 4, stride 2, 'same' padding, BatchNorm, LeakyReLU

* **Output:** A single value indicating whether the input image is real or fake.

Let's start implementing these networks in code! 

In [ ]:
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.layers import BatchNormalization, Flatten, Input
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Model


def get_generator():
    input_noise = Input(shape=(1, 1, 100))

    # ====== Block 1 ======
    x = Conv2DTranspose(512, kernel_size=4, strides=1, padding="valid")(input_noise)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    # =====================

    # ====== Block 2 ======
    # TODO
    # =====================

    # ====== Block 3 ======
    # TODO
    # =====================

    x = Conv2DTranspose(1, kernel_size=4, strides=2, padding="same", activation="tanh")(x)

    return Model(inputs=input_noise, outputs=x)


def get_discriminator():
    input_image = Input(shape=(16, 16, 1))

    # ====== Block 1 ======
    x = Conv2D(128, kernel_size=4, strides=2, padding="same")(input_image)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    # =====================

    # ====== Block 2 ======
    # TODO
    # =====================

    x = Conv2D(1, kernel_size=4, strides=1, padding="valid")(x)
    x = Flatten()(x)

    return Model(inputs=input_image, outputs=x)

generator = get_generator()
discriminator = get_discriminator()

Now that our models are created, a sanity check must be done on the output dimension.

In [ ]:
def get_noise(batch_size, nz=100):
    return tf.random.normal([batch_size, 1, 1, nz])

noise = get_noise(20)

print("init", noise.shape)
fake_images = generator(noise)
print("Fake images", fake_images.shape)  # Should be (_, 64, 64, 1)
preds = discriminator(fake_images)
print("Predictions", preds.shape)  # Should be (_, 1)

In [ ]:
generator.summary()
discriminator.summary()

## Understanding Loss Functions in DCGAN

DCGAN employs two distinct loss functions to guide the training process:

**1. Discriminator Loss:**

* **Goal:**  The discriminator needs to accurately distinguish between real images (labeled as 'real') and generated images (labeled as 'fake'). 
* **Loss Function:** We'll use `binary_crossentropy` with the `from_logits=True` option. This combines the sigmoid activation (for probability calculation) directly within the loss calculation.

**2. Generator Loss:**

* **Goal:** The generator aims to deceive the discriminator into classifying its generated images as 'real'.
* **Loss Function:**  Again, we can use `binary_crossentropy`  with `from_logits=True`. 

**Let's Implement These Losses:**

In [ ]:
from tensorflow.keras.losses import binary_crossentropy

def discriminator_loss(preds_real, preds_fake):
    loss_real = binary_crossentropy(tf.ones_like(preds_real), preds_real, from_logits=True)
    loss_fake = binary_crossentropy(tf.zeros_like(preds_fake), preds_fake, from_logits=True)
    return loss_real + loss_fake

def generator_loss(preds_fake):
    return binary_crossentropy(tf.ones_like(preds_fake), preds_fake, from_logits=True)

We create two optimizers, one for the discriminator and one for the generator. Remember that Adam maintains internal state for these parameters in addition to updating them. This internal state consists of "m" and "v" vectors, which are updated on each iteration. For this reason, we need to create two optimizers, one for the generator and one for the discriminator, so that we can update them separately.

In [ ]:
from tensorflow.keras.optimizers import Adam

discriminator_optimizer = Adam()
generator_optimizer = Adam()

We define our train step for a single batch. Because we are using a fairly complex architecture, we can't rely on tensorflow to automatically `fit` our model. We need to define our training loop. This method uses the `@tf.function` decorator to convert the Python function into a TensorFlow graph function. This will speed up the training process.

We also use `tf.GradientTape` to record the operations for automatic differentiation. This tells TensorFlow to record the operations in the forward pass and then compute the gradients in the backward pass.

In [ ]:
@tf.function
def train_step(images):
    """Performs a single training step for the DCGAN.

    Args:
        images: A batch of real images.

    Returns:
        disc_loss: The discriminator's loss for this step.
        gen_loss: The generator's loss for this step.
    """
    batch_size = images.shape[0]
    noise = get_noise(batch_size)

    # Train the discriminator
    with tf.GradientTape() as disc_tape:
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generator(noise, training=True), training=True)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    # Train the generator
    with tf.GradientTape() as gen_tape:
        generated_images = generator(noise, training=True)
        fake_output = discriminator(generated_images, training=True)
        gen_loss = generator_loss(fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))

    return disc_loss, gen_loss


## Initiating DCGAN Training

Now it's time to train your DCGAN! Here's what to keep in mind:

* **Monitoring Losses:**  Track both discriminator and generator losses during training.  Ideally, you want to see a balance; if one loss consistently becomes too low while the other rises sharply, it indicates an imbalance in the adversarial training dynamic.

* **Visualizing Progress:**  Periodically generate images using your generator. You should see realistic-looking digits emerge fairly quickly (within a few epochs). Expect finer details and overall quality to improve as training continues.

**Important Note:** Reaching a perfect convergence where losses stabilize around specific values  might be less common in GAN training compared to some other deep learning tasks. The focus is often on the quality of generated samples and  achieving balance in the adversarial training process. 

In [ ]:
from tensorflow.keras.metrics import Mean
from IPython.display import display as jupy_display

epochs = 10
fixed_noise = get_noise(20)

print("Initial Generated Images (using base noise):")
fake_images = generator(fixed_noise, training=False).numpy()
jupy_display(display_images(fake_images))

for epoch in range(epochs):
    print("====== Epoch {:2d} ======".format(epoch))

    epoch_loss_d = Mean()
    epoch_loss_g = Mean()

    # Progress Indicator
    batch_count = tf.data.experimental.cardinality(data_generator).numpy()
    print(f"Number of batches per epoch: {batch_count}")

    for step, real_images in enumerate(data_generator):
        loss_d, loss_g = train_step(real_images)
        epoch_loss_d(loss_d)
        epoch_loss_g(loss_g)

        if step % 20 == 0:
            print(f"--> ", end="")

    print("\nEpoch Summary:")
    print(f"  Discriminator Loss: {epoch_loss_d.result():.4f}")
    print(f"  Generator Loss:     {epoch_loss_g.result():.4f}")

    #  Track Generated Images with the Same Noise 
    fake_images = generator(fixed_noise, training=False).numpy()
    jupy_display(display_images(fake_images))

In [ ]:
# Display Final Generated Images

print("Final Generated Images (using base noise):")
fake_images = generator(fixed_noise, training=False).numpy()
jupy_display(display_images(fake_images))

## Wrapping Up: Your DCGAN Journey

Congratulations on training your DCGAN! You've grasped the fundamentals of how GANs can generate realistic images. 

###  Explore and Experiment

If you'd like to take your DCGAN further, here are some potential projects to try:

* **Play with Noise:**  See how different shapes and distributions of your input noise influence the images your generator creates. Can you generate patterns or textures instead of realistic digits?

* **Tweak the Architecture:**  Try adding or removing a convolutional layer in your generator or discriminator. Observe how this affects the quality and detail of the generated images.

* **Introduce Color:**  Adapt your DCGAN to generate color images (RGB)  instead of grayscale. You'll need to adjust the output channels of your generator.

* **Visualize the Training:**  Plot the discriminator and generator losses over time using a tool like Matplotlib or TensorBoard.  This can help you diagnose training issues.

**Remember,  learning about GANs often involves  experimentation and seeing what works best. Have fun exploring!**
